<a href="https://colab.research.google.com/github/sumanjitraut/MyFirstGit/blob/main/VIBE_CODING_Claim_Insight.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Vibe Coding
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# Synthetic daily claims data for 200 days
np.random.seed(42)
days = 200
claims = np.random.poisson(lam=50, size=days) + np.sin(np.linspace(0, 3*np.pi, days)) * 10

# Convert to DataFrame
df = pd.DataFrame({'claims': claims})
df.plot(title='Synthetic Daily Claims')
#print(df)

In [ ]:
# Scale between 0 and 1
scaler = MinMaxScaler()
scaled_claims = scaler.fit_transform(df[['claims']])
#print(scaled_claims)
# Create sequences (window = 10 days)
def create_sequences(data, window):
    X, y = [], []
    for i in range(len(data) - window):
        X.append(data[i:i+window])
        y.append(data[i+window])
    return np.array(X), np.array(y)

window = 10
X, y = create_sequences(scaled_claims, window)
#print(y)
# Reshape for LSTM: (samples, time steps, features)
X = X.reshape((X.shape[0], window, 1))
print(X)

In [ ]:
#build the LSTM
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(window, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=30, verbose=1)


In [ ]:
# Predict future claims
predicted = model.predict(X)
predicted_claims = scaler.inverse_transform(predicted)
actual_claims = scaler.inverse_transform(y)

# Plot predictions vs actual
plt.plot(actual_claims, label='Actual')
plt.plot(predicted_claims, label='Predicted')
plt.title("Claim Forecasting with LSTM")
plt.legend()
plt.show()

In [ ]:
#Future with error

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Input

# Step 1: Generate synthetic claim data
np.random.seed(42)
days = 200
claims = np.random.poisson(lam=50, size=days) + np.sin(np.linspace(0, 3*np.pi, days)) * 10
df = pd.DataFrame({'claims': claims})

# Step 2: Scale data between 0 and 1
scaler = MinMaxScaler()
scaled_claims = scaler.fit_transform(df[['claims']])

# Step 3: Create sequences for LSTM
def create_sequences(data, window):
    X, y = [], []
    for i in range(len(data) - window):
        X.append(data[i:i+window])
        y.append(data[i+window])
    return np.array(X), np.array(y)

window = 10
X, y = create_sequences(scaled_claims, window)
X = X.reshape((X.shape[0], window, 1))

# Step 4: Build and train the LSTM model
model = Sequential()
model.add(Input(shape=(window, 1)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit(X, y, epochs=30, verbose=0)

# Step 5: Predict values on training data
predicted = model.predict(X)
predicted_claims = scaler.inverse_transform(predicted)
actual_claims = scaler.inverse_transform(y)

# Step 6: Error Metrics
mae = mean_absolute_error(actual_claims, predicted_claims)
rmse = np.sqrt(mean_squared_error(actual_claims, predicted_claims))
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")

# Step 7: Forecast 21 future days
future_input = X[-1]  # last known input sequence
future_preds = []

for _ in range(21):
    input_reshaped = future_input.reshape((1, window, 1))
    next_pred_scaled = model.predict(input_reshaped)
    next_pred = scaler.inverse_transform(next_pred_scaled)
    future_preds.append(next_pred[0][0])

    # update input sequence by dropping oldest and appending newest
    future_input = np.append(future_input[1:], next_pred_scaled, axis=0)

# Step 8: Plot results
plt.figure(figsize=(12, 5))
plt.plot(actual_claims.flatten(), label='Actual Claims')
plt.plot(predicted_claims.flatten(), label='Predicted Claims')
plt.plot(range(len(predicted_claims), len(predicted_claims) + 21), future_preds, label='Future 21-Day Forecast', linestyle='--')
plt.axvline(x=len(predicted_claims)-1, color='gray', linestyle=':', label='Forecast Starts')
plt.title("LSTM Claim Forecasting with 21-Day Future Extension")
plt.xlabel("Days")
plt.ylabel("Claim Volume")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()
